## Aggregate Results

### Self-Truncation

In [ ]:
# 1. check outputsteps
import glob

folder = 'edit'

# result_file_list = glob.glob(f"results/{folder}/gsm8k*")
result_file_list = glob.glob(f"results/{folder}/math500*")
# result_file_list = glob.glob(f"results/{folder}/aimo*")
# result_file_list = glob.glob(f"results/{folder}/aimo_r1*")
result_file_list

### Dual Goal Search

In [ ]:
import json
import pandas as pd
import re, os
from mapping import modelname2hfpath
from collections import Counter
from utils import extract_last_boxed
from transformers import AutoTokenizer
from collections import defaultdict
from tqdm import tqdm

"""
{
    "<id>": {
        "question": str,
        "result_list": [
            {
                "output_solution": [
                    {'text': solution 1, 'cum_logprob': float},
                    {'text': solution 2, 'cum_logprob': float}
                    ...
                ],
                "max_steps": int
            }   # solutions and metadata at constriant 1
            {
                "output_solution": [
                    solution 1,
                    solution 2,
                ],
                "max_steps": int
            }   # solution at constriant 2
        ]
        "pred_num": int,
        "answer": str,
    }
}
"""

for res_idx in range(len(result_file_list)):
# for res_idx in range(2):
    filename = os.path.basename(result_file_list[res_idx])
    metadata = filename.replace('.jsonl', '').split("_")
    dataset, modelname = metadata[0], metadata[1]
    tknz = AutoTokenizer.from_pretrained(modelname2hfpath[modelname])
    total_length = 0
    total_length_pen = 0
    correct = total = 0

    res_df = pd.read_json(result_file_list[res_idx], lines=True)
    # res_df = pd.read_json(result_file_list[res_idx])
    # iterate over samples
    for i, group_rows in res_df.groupby('question'):
        # for all solutions with given constraints
        min_correct_length = 10000
        max_wrong_length = 0

        is_correct = False
        for idx, row in group_rows.iterrows():
            per_step_solutions = row['output_solution']
            max_steps = row.get('max_steps', 0)
            if dataset == "gsm8k":
                answer = re.findall(r"#### (-?\d+)", row["answer"])[-1]
            else:
                answer = row["answer"]

            cnter = Counter()
            answer_idx = defaultdict(list)
            
            for sol_idx, solution in enumerate(per_step_solutions):
                if isinstance(solution, dict):
                    solution = solution['text']
                pred_answer = extract_last_boxed(solution)
                if pred_answer is None:
                    continue
                cnter[pred_answer] += 1
                answer_idx[pred_answer].append(sol_idx)
        
            if len(cnter) > 0:
                sc_answer = cnter.most_common(1)[0][0]
            else:
                sc_answer = None
            
            if sc_answer == str(answer):
                lengths = []
                for idx in answer_idx[sc_answer]:
                    lengths.append(len(tknz.encode(per_step_solutions[idx])))
                
                # total_length_pen += max(lengths)

                min_correct_length = min(min_correct_length, min(lengths))

                # check if under any constraint we got the answer
                is_correct = True
            else:
                # total_length_pen += max(len(tknz.encode(sol)) for sol in per_step_solutions["output_solution"]) * 2
                max_wrong_length = max(max_wrong_length, max(len(tknz.encode(sol)) for sol in per_step_solutions) * 2)
            
        if is_correct:
            correct += 1
            total_length += min_correct_length
            total_length_pen += min_correct_length
        else:
            total_length_pen += max_wrong_length

        total += 1

    print(f"Model: {modelname}\n correct: {correct}, total: {total}, max_step: {max_steps}\n correct rate: {correct / total*100:.2f}%, avg_length: {total_length / total}, avg_length_pen: {total_length_pen / total}")